<a href="https://colab.research.google.com/github/luminol-io/hub/blob/gh-pages/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from luminol import *



---


In [2]:
path = grab("nietzsche.txt",
            "https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [3]:
text = data(path,
            action = 'read',
            encode_type = 'utf-8',
            process = ['lower', 'remove_newline_to_empty_space'])

In [7]:
chars = sorted(list(set(text)))
print("Total chars:", len(chars))

Total chars: 56


In [11]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

---

In [12]:
maxlen = 40
step = 3
sentences = []
next_chars = []

In [13]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])

print("Number of sequences:", len(sentences))

Number of sequences: 200285


In [14]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [17]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



---



In [29]:
model = keras.Sequential(
    [
        keras.Input(shape = (maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation = "softmax"),
    ]
)

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)

In [30]:
model.compile(loss="categorical_crossentropy", optimizer=optimizer)



---



In [31]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



---



In [32]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

1565/1565 [==============================] - 196s 124ms/step - loss: 1.9111

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "arly "immoral" act, so that to-day we as"
...Generated:   a thes has instinct and the one superst the contration of experience of elivide of the really the for a the the something the respect, in the respection of the respection of the religious est the religious and the respection of the will the perion of instinct of the something the religious and instinct of the the religious and the lates and the something the morality of the religious of all the s

...Diversity: 0.5
...Generating with seed: "arly "immoral" act, so that to-day we as"
...Generated:   it become into hist teres experience of ascoust to them as as theme proises, and the created and are consicted and allature of elition are a something and the something the will the lary the streng of all the cousting the fature until any amus and its suphich as en only underst peasity 

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f2ca1539200>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


...Generated:  ll therewfore operate philosophers. altrough snepies, plasm tormenta."" withes, no such aderatude of xease a rayer be other nowed tajnimances lainualipties are even innocent, .k in now to the mod in the morality for breasity, and treatically advance, slown su mwants, nowadayh everythilitation itsal, in fact of the emply befope-wied.=--neas no "nothing self, and own, still e motans ejociplid beer: 

...Diversity: 1.2
...Generating with seed: "ay and battle of forces. that which is a"


KeyboardInterrupt: ignored